In [1]:
import os
import pathlib

while pathlib.Path(os.curdir).resolve().name != "DeepAWB":
    os.chdir("..")

%load_ext autoreload
%autoreload 2

In [15]:
from src import RAW_TEST_IMAGES_FOLDER, RAW_TRAIN_IMAGES_FOLDER
from src.bayesian_awb.bayesian_awb_algorithm import perform_bayes_AWB
from src.deep_awb.data_loaders import SimpleCubePPDatasetInfo, get_test_dataset, get_train_dataset

SimpleCubePPDatasetInfo.setup()
annotations = get_train_dataset().annotations
image_names = annotations["image"].values + ".png"
gt_gains = annotations[["R/G", "B/G"]].values

2024-09-21 15:15:37.706 | WARNING  | src.deep_awb.data_loaders:setup:101 - Called `setup` with image_scale=1 when it was already set to self._image_scale=1


In [3]:
import pathlib

import tqdm.notebook as tqdm


def compute_average_MSE_loss(image_paths: list[pathlib.Path], gt_gains, min_avg_G, max_delta_squared) -> float:
    assert len(image_paths) == len(gt_gains)
    MSE_total = 0
    for i in tqdm.trange(len(image_paths)):
        image_path, gt_gain = image_paths[i], gt_gains[i]
        r_gain, b_gain = perform_bayes_AWB(image_path, min_avg_G, max_delta_squared)
        MSE_total += (r_gain - gt_gain[0]) ** 2 + (b_gain - gt_gain[1]) ** 2
    return MSE_total / len(image_paths)

In [4]:
import pandas as pd

image_paths = [RAW_TRAIN_IMAGES_FOLDER / image_name for image_name in image_names]

experiments = pd.DataFrame(columns=["min_avg_G", "max_delta_squared", "MSE_loss"])

for min_avg_G in [0.0001, 0.001, 0.005]:
    for max_delta_squared in [0.1, 0.15, 0.25, 0.35]:
        MSE_loss = compute_average_MSE_loss(image_paths, gt_gains, min_avg_G, max_delta_squared)
        experiments.loc[len(experiments)] = [min_avg_G, max_delta_squared, MSE_loss]

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

...

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/1772 [00:00<?, ?it/s]

In [10]:
experiments.sort_values("MSE_loss")

,min_avg_G,max_delta_squared,MSE_loss
8,0.0050,0.10,0.011750
11,0.0050,0.35,0.011802
10,0.0050,0.25,0.011803
4,0.0010,0.10,0.011829
0,0.0001,0.10,0.011832
6,0.0010,0.25,0.011890
7,0.0010,0.35,0.011895
9,0.0050,0.15,0.011897
2,0.0001,0.25,0.011900
3,0.0001,0.35,0.011901


### The best hyperparameters are `min_avg_G` = $0.005$ and `max_delta_squared` = $0.1$.


In [16]:
min_avg_G, max_delta_squared = 0.005, 0.1

annotations = get_test_dataset().annotations
image_names = annotations["image"].values + ".png"
gt_gains = annotations[["R/G", "B/G"]].values

MSE_loss = compute_average_MSE_loss([RAW_TEST_IMAGES_FOLDER / image_name for image_name in image_names], gt_gains, min_avg_G, max_delta_squared)

  0%|          | 0/462 [00:00<?, ?it/s]

# Test MSE loss:


In [19]:
MSE_loss

0.014168253541418936